In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Capsule/Dynamic _rounting_enhancement_Capsule_network

In [ ]:
!pip install tensorflow==1.14.0 
!pip install keras==2.1.5

# Dependencies

In [ ]:


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import h5py
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import keras
from keras import backend as K
from tensorflow.contrib.layers.python.layers import initializers

import collections
import pickle
import gensim
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import re
import random
import sys
import os 
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import array
from numpy import asarray
from numpy import zeros
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.layers import Dropout, Activation, Flatten,Embedding, Convolution1D, MaxPooling1D, AveragePooling1D, Input, Dense, merge,Add
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.datasets import imdb
from keras import callbacks
from keras.utils import generic_utils
from keras.models import Model
from keras.optimizers import Adadelta
import time

import numpy as np
import pickle
from collections import defaultdict
import sys, re
import pandas as pd

from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import string

from string import punctuation
from os import listdir
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import nltk
nltk.download('stopwords')

import tensorflow as tf
print(tf.__version__)

# Model

In [ ]:
class Classifer(keras.layers.Layer):

    def get_hidden_states_before(self, hidden_states, step, shape, hidden_size):
        #padding zeros
        padding=tf.zeros((shape[0], step, hidden_size), dtype=tf.float32)
        #remove last steps
        displaced_hidden_states=hidden_states[:,:-step,:]
        #concat padding
        return tf.concat([padding, displaced_hidden_states], axis=1)
        #return tf.cond(step<=shape[1], lambda: tf.concat([padding, displaced_hidden_states], axis=1), lambda: tf.zeros((shape[0], shape[1], self.config.hidden_size_sum), dtype=tf.float32))

    def get_hidden_states_after(self, hidden_states, step, shape, hidden_size):
        #padding zeros
        padding=tf.zeros((shape[0], step, hidden_size), dtype=tf.float32)
        #remove last steps
        displaced_hidden_states=hidden_states[:,step:,:]
        #concat padding
        return tf.concat([displaced_hidden_states, padding], axis=1)
        #return tf.cond(step<=shape[1], lambda: tf.concat([displaced_hidden_states, padding], axis=1), lambda: tf.zeros((shape[0], shape[1], self.config.hidden_size_sum), dtype=tf.float32))

    def sum_together(self, l):
        combined_state=None
        for tensor in l:
            if combined_state==None:
                combined_state=tensor
            else:
                combined_state=combined_state+tensor
        return combined_state
    
    def slstm_cell(self, name_scope_name, hidden_size, lengths, initial_hidden_states, initial_cell_states, num_layers):
        with tf.name_scope(name_scope_name):
            #Word parameters 
            #forget gate for left 
            with tf.name_scope("f1_gate"):
                #current
                Wxf1 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                #left right
                Whf1 = tf.Variable(tf.random_normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                #initial state
                Wif1 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                #dummy node
                Wdf1 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #forget gate for right 
            with tf.name_scope("f2_gate"):
                Wxf2 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                Whf2 = tf.Variable(tf.random_normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                Wif2 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                Wdf2 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #forget gate for inital states     
            with tf.name_scope("f3_gate"):
                Wxf3 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                Whf3 = tf.Variable(tf.random_normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                Wif3 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                Wdf3 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #forget gate for dummy states     
            with tf.name_scope("f4_gate"):
                Wxf4 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                Whf4 = tf.Variable(tf.random_normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
                Wif4 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wif")
                Wdf4 = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdf")
            #input gate for current state     
            with tf.name_scope("i_gate"):
                Wxi = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxi")
                Whi = tf.Variable(tf.random_normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whi")
                Wii = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wii")
                Wdi = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdi")
            #input gate for output gate
            with tf.name_scope("o_gate"):
                Wxo = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
                Who = tf.Variable(tf.random_normal([2*hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
                Wio = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wio")
                Wdo = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wdo")
            #bias for the gates    
            with tf.name_scope("biases"):
                bi = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bi")
                bo = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
                bf1 = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf1")
                bf2 = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf2")
                bf3 = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf3")
                bf4 = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bf4")

            #dummy node gated attention parameters
            #input gate for dummy state
            with tf.name_scope("gated_d_gate"):
                gated_Wxd = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxf")
                gated_Whd = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Whf")
            #output gate
            with tf.name_scope("gated_o_gate"):
                gated_Wxo = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
                gated_Who = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
            #forget gate for states of word
            with tf.name_scope("gated_f_gate"):
                gated_Wxf = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Wxo")
                gated_Whf = tf.Variable(tf.random_normal([hidden_size, hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="Who")
            #biases
            with tf.name_scope("gated_biases"):
                gated_bd = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bi")
                gated_bo = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
                gated_bf = tf.Variable(tf.random_normal([hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="bo")
        # print("first phase done!")
        #filters for attention        
        mask_softmax_score=tf.cast(tf.sequence_mask(lengths), tf.float32)*1e25-1e25
        # print("second phase done!")
        # print(mask_softmax_score.shape)
        mask_softmax_score_expanded=tf.expand_dims(mask_softmax_score, axis=2)
        # print("third phase done!")               
        #filter invalid steps
        sequence_mask=tf.expand_dims(tf.cast(tf.sequence_mask(lengths), tf.float32),axis=2)
        # print("fourth phase done!")
        # print(initial_hidden_states.shape)
        # print(sequence_mask.shape)
        #filter embedding states
        initial_hidden_states=initial_hidden_states*sequence_mask
        # print("fifth phase done!")
        initial_cell_states=initial_cell_states*sequence_mask
        # print("sixth phase done!")
        #record shape of the batch
        shape=tf.shape(initial_hidden_states)
        
        #initial embedding states
        embedding_hidden_state=tf.reshape(initial_hidden_states, [-1, hidden_size])      
        embedding_cell_state=tf.reshape(initial_cell_states, [-1, hidden_size])

        #randomly initialize the states
        if config.random_initialize:
            initial_hidden_states=tf.random_uniform(shape, minval=-0.05, maxval=0.05, dtype=tf.float32, seed=None, name=None)
            initial_cell_states=tf.random_uniform(shape, minval=-0.05, maxval=0.05, dtype=tf.float32, seed=None, name=None)
            #filter it
            initial_hidden_states=initial_hidden_states*sequence_mask
            initial_cell_states=initial_cell_states*sequence_mask

        #inital dummy node states
        dummynode_hidden_states=tf.reduce_mean(initial_hidden_states, axis=1)
        dummynode_cell_states=tf.reduce_mean(initial_cell_states, axis=1)

        for i in range(num_layers):
            #update dummy node states
            #average states
            combined_word_hidden_state=tf.reduce_mean(initial_hidden_states, axis=1)
            reshaped_hidden_output=tf.reshape(initial_hidden_states, [-1, hidden_size])
            #copy dummy states for computing forget gate
            transformed_dummynode_hidden_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_hidden_states, axis=1), [1, shape[1],1]), [-1, hidden_size])
            #input gate
            gated_d_t = tf.nn.sigmoid(
                tf.matmul(dummynode_hidden_states, gated_Wxd) + tf.matmul(combined_word_hidden_state, gated_Whd) + gated_bd
            )
            #output gate
            gated_o_t = tf.nn.sigmoid(
                tf.matmul(dummynode_hidden_states, gated_Wxo) + tf.matmul(combined_word_hidden_state, gated_Who) + gated_bo
            )
            #forget gate for hidden states
            gated_f_t = tf.nn.sigmoid(
                tf.matmul(transformed_dummynode_hidden_states, gated_Wxf) + tf.matmul(reshaped_hidden_output, gated_Whf) + gated_bf
            )

            #softmax on each hidden dimension 
            reshaped_gated_f_t=tf.reshape(gated_f_t, [shape[0], shape[1], hidden_size])+ mask_softmax_score_expanded
            gated_softmax_scores=tf.nn.softmax(tf.concat([reshaped_gated_f_t, tf.expand_dims(gated_d_t, dim=1)], axis=1), dim=1)
            #split the softmax scores
            new_reshaped_gated_f_t=gated_softmax_scores[:,:shape[1],:]
            new_gated_d_t=gated_softmax_scores[:,shape[1]:,:]
            #new dummy states
            dummy_c_t=tf.reduce_sum(new_reshaped_gated_f_t * initial_cell_states, axis=1) + tf.squeeze(new_gated_d_t, axis=1)*dummynode_cell_states
            dummy_h_t=gated_o_t * tf.nn.tanh(dummy_c_t)

            #update word node states
            #get states before
            initial_hidden_states_before=[tf.reshape(self.get_hidden_states_before(initial_hidden_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_hidden_states_before=self.sum_together(initial_hidden_states_before)
            initial_hidden_states_after= [tf.reshape(self.get_hidden_states_after(initial_hidden_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_hidden_states_after=self.sum_together(initial_hidden_states_after)
            #get states after
            initial_cell_states_before=[tf.reshape(self.get_hidden_states_before(initial_cell_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_cell_states_before=self.sum_together(initial_cell_states_before)
            initial_cell_states_after=[tf.reshape(self.get_hidden_states_after(initial_cell_states, step+1, shape, hidden_size), [-1, hidden_size]) for step in range(self.config.step)]
            initial_cell_states_after=self.sum_together(initial_cell_states_after)
            
            #reshape for matmul
            initial_hidden_states=tf.reshape(initial_hidden_states, [-1, hidden_size])
            initial_cell_states=tf.reshape(initial_cell_states, [-1, hidden_size])

            #concat before and after hidden states
            concat_before_after=tf.concat([initial_hidden_states_before, initial_hidden_states_after], axis=1)

            #copy dummy node states 
            transformed_dummynode_hidden_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_hidden_states, axis=1), [1, shape[1],1]), [-1, hidden_size])
            transformed_dummynode_cell_states=tf.reshape(tf.tile(tf.expand_dims(dummynode_cell_states, axis=1), [1, shape[1],1]), [-1, hidden_size])

            f1_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf1) + tf.matmul(concat_before_after, Whf1) + 
                tf.matmul(embedding_hidden_state, Wif1) + tf.matmul(transformed_dummynode_hidden_states, Wdf1)+ bf1
            )

            f2_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf2) + tf.matmul(concat_before_after, Whf2) + 
                tf.matmul(embedding_hidden_state, Wif2) + tf.matmul(transformed_dummynode_hidden_states, Wdf2)+ bf2
            )

            f3_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf3) + tf.matmul(concat_before_after, Whf3) + 
                tf.matmul(embedding_hidden_state, Wif3) + tf.matmul(transformed_dummynode_hidden_states, Wdf3) + bf3
            )

            f4_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxf4) + tf.matmul(concat_before_after, Whf4) + 
                tf.matmul(embedding_hidden_state, Wif4) + tf.matmul(transformed_dummynode_hidden_states, Wdf4) + bf4
            )
            
            i_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxi) + tf.matmul(concat_before_after, Whi) + 
                tf.matmul(embedding_hidden_state, Wii) + tf.matmul(transformed_dummynode_hidden_states, Wdi)+ bi
            )
            
            o_t = tf.nn.sigmoid(
                tf.matmul(initial_hidden_states, Wxo) + tf.matmul(concat_before_after, Who) + 
                tf.matmul(embedding_hidden_state, Wio) + tf.matmul(transformed_dummynode_hidden_states, Wdo) + bo
            )
            
            f1_t, f2_t, f3_t, f4_t, i_t=tf.expand_dims(f1_t, axis=1), tf.expand_dims(f2_t, axis=1),tf.expand_dims(f3_t, axis=1), tf.expand_dims(f4_t, axis=1), tf.expand_dims(i_t, axis=1)


            five_gates=tf.concat([f1_t, f2_t, f3_t, f4_t,i_t], axis=1)
            five_gates=tf.nn.softmax(five_gates, dim=1)
            f1_t,f2_t,f3_t, f4_t,i_t= tf.split(five_gates, num_or_size_splits=5, axis=1)
            
            f1_t, f2_t, f3_t, f4_t, i_t=tf.squeeze(f1_t, axis=1), tf.squeeze(f2_t, axis=1),tf.squeeze(f3_t, axis=1), tf.squeeze(f4_t, axis=1),tf.squeeze(i_t, axis=1)

            c_t = (f1_t * initial_cell_states_before) + (f2_t * initial_cell_states_after)+(f3_t * embedding_cell_state)+ (f4_t * transformed_dummynode_cell_states)+ (i_t * initial_cell_states)
            
            h_t = o_t * tf.nn.tanh(c_t)

            #update states
            initial_hidden_states=tf.reshape(h_t, [shape[0], shape[1], hidden_size])
            initial_cell_states=tf.reshape(c_t, [shape[0], shape[1], hidden_size])
            initial_hidden_states=initial_hidden_states*sequence_mask
            initial_cell_states=initial_cell_states*sequence_mask

            dummynode_hidden_states=dummy_h_t
            dummynode_cell_states=dummy_c_t

        initial_hidden_states = tf.nn.dropout(initial_hidden_states,self.dropout)
        initial_cell_states = tf.nn.dropout(initial_cell_states, self.dropout)

        return initial_hidden_states, initial_cell_states, dummynode_hidden_states


    def slstm_basic_layer(self,embedding, config, mask):
        embedding = tf.squeeze(embedding)
        initial_hidden_states = embedding
        # print("embedding shape")
        # print(embedding.shape)
        initial_cell_states=tf.identity(initial_hidden_states)

        initial_hidden_states = tf.nn.dropout(initial_hidden_states,config.keep_prob)
        # print("initial shape")
        # print(initial_hidden_states.shape)
        initial_cell_states = tf.nn.dropout(initial_cell_states, config.keep_prob)

        #create layers 
        
        new_hidden_states,new_cell_state, dummynode_hidden_states = self.slstm_cell("word_slstm", config.hidden_size, mask, initial_hidden_states, initial_cell_states, config.layer)
        
        softmax_w = tf.Variable(tf.random_normal([2*config.hidden_size, config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w")
        softmax_b = tf.Variable(tf.random_normal([config.num_label], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b")
        #representation=dummynode_hidden_states
        representation=tf.reduce_mean(tf.concat([new_hidden_states, tf.expand_dims(dummynode_hidden_states, axis=1)], axis=1), axis=1)
        representation1=tf.reduce_mean(tf.concat([new_hidden_states, tf.expand_dims(dummynode_hidden_states, axis=1)], axis=1), axis=1)
        # new_hidden_states_concat = tf.concat([new_hidden_states, tf.expand_dims(dummynode_hidden_states, axis=1)], axis=1)
        softmax_w2 = tf.Variable(tf.random_normal([config.hidden_size, 2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w2")
        softmax_b2 = tf.Variable(tf.random_normal([2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b2")
        softmax_w22 = tf.Variable(tf.random_normal([config.hidden_size, 2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_w2")
        softmax_b22 = tf.Variable(tf.random_normal([2*config.hidden_size], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32, name="softmax_b2")
        representation=tf.nn.tanh(tf.matmul(representation, softmax_w2)+softmax_b2)
        representation1=tf.nn.tanh(tf.matmul(representation1, softmax_w22)+softmax_b22)
        
        return new_hidden_states, representation, representation1 

    def __init__(self, config):
        self.config=config
        self.dropout=self.keep_prob=config.keep_prob

  

In [ ]:
def _matmul_broadcast(x, y, name):
  """Compute x @ y, broadcasting over the first `N - 2` ranks.
  """
  with tf.variable_scope(name) as scope:
    return tf.reduce_sum(
      tf.nn.dropout(x[..., tf.newaxis] * y[..., tf.newaxis, :, :],1), axis=-2
    )


In [ ]:
z = tf.constant(0.1, shape=[2,3,4])

In [ ]:
z[...,tf.newaxis]

In [ ]:
def _conv2d_wrapper(inputs, shape, strides, padding, add_bias, activation_fn, name, stddev=0.1):
  """Wrapper over tf.nn.conv2d().
  """

  with tf.variable_scope(name) as scope:
    kernel = _get_weights_wrapper(
      name='weights', shape=shape, weights_decay_factor=0.0, )
    output = tf.nn.conv2d(inputs, filter=kernel, strides=strides, padding=padding, name='conv')
    if add_bias:
      biases = _get_biases_wrapper(name='biases', shape=[shape[-1]] )
      output = tf.add(output, biases, name='biasAdd')
    if activation_fn is not None:
      output = activation_fn(output, name='activation')
  return output

In [ ]:
def _get_weights_wrapper(name, shape, dtype=tf.float32, initializer=initializers.xavier_initializer(),weights_decay_factor=None):
  """Wrapper over _get_variable_wrapper() to get weights, with weights decay factor in loss.
  """

  weights = _get_variable_wrapper(name=name, shape=shape, dtype=dtype, initializer=initializer)

  if weights_decay_factor is not None and weights_decay_factor > 0.0:
    weights_wd = tf.multiply(tf.nn.l2_loss(weights), weights_decay_factor, name=name + '/l2loss')
    tf.add_to_collection('losses', weights_wd)

  return weights

In [ ]:
def _get_biases_wrapper(name, shape, dtype=tf.float32, initializer=tf.constant_initializer(0.0)):
  """Wrapper over _get_variable_wrapper() to get bias.
  """
  biases = _get_variable_wrapper(name=name, shape=shape, dtype=dtype, initializer=initializer)
  return biases

In [ ]:
def _get_variable_wrapper(
  name, shape=None, dtype=None, initializer=None,
  regularizer=None,
  trainable=True,
  collections=None,
  caching_device=None,
  partitioner=None,
  validate_shape=True,
  custom_getter=None
):
  """Wrapper over tf.get_variable().
  """

  with tf.device('/cpu:0'):
    var = tf.get_variable(
      name, shape=shape, dtype=dtype, initializer=initializer,
      regularizer=regularizer, trainable=trainable,
      collections=collections, caching_device=caching_device,
      partitioner=partitioner, validate_shape=validate_shape,
      custom_getter=custom_getter
    )
  return var

In [ ]:
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex/K.sum(ex, axis=axis, keepdims=True)

In [ ]:
def squash_v1(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    return scale * x

In [ ]:
def squash_v0(s, axis=-1, epsilon=1e-7, name=None):
    s_squared_norm = K.sum(K.square(s), axis, keepdims=True) + K.epsilon()
    safe_norm = K.sqrt(s_squared_norm)
    scale = 1 - tf.exp(-safe_norm)
    return scale * s / safe_norm

In [ ]:
def routing1(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations, context):
    b = keras.backend.zeros_like(u_hat_vecs[:,:,:,0])+ context
   
    if i_activations is not None:
        i_activations = i_activations[...,tf.newaxis]
    for i in range(iterations):
        if False:
            leak = tf.zeros_like(b, optimize=True)
            leak = tf.reduce_sum(leak, axis=1, keep_dims=True)
            leaky_logits = tf.concat([leak, b], axis=1)
            leaky_routing = tf.nn.softmax(leaky_logits, dim=1)        
            c = tf.split(leaky_routing, [1, output_capsule_num], axis=1)[1]
        else:
            c = softmax(b, 1)   
#        if i_activations is not None:
#            tf.transpose(tf.transpose(c, perm=[0,2,1]) * i_activations, perm=[0,2,1]) 
        outputs = squash_v1(K.batch_dot(c, u_hat_vecs, [2, 2]))
        if i < iterations - 1:
            b = b + K.batch_dot(outputs, u_hat_vecs, [2, 3])                                    
    poses = outputs 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations


In [ ]:
def routing(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations, context_sensitivity):
    b = keras.backend.zeros_like(u_hat_vecs[:,:,:,0]) + context_sensitivity
    
    if i_activations is not None:
        i_activations = i_activations[...,tf.newaxis]
    for i in range(iterations):
        if False:
            leak = tf.zeros_like(b, optimize=True)
            leak = tf.reduce_sum(leak, axis=1, keep_dims=True)
            leaky_logits = tf.concat([leak, b], axis=1)
            leaky_routing = tf.nn.softmax(leaky_logits, dim=1)        
            c = tf.split(leaky_routing, [1, output_capsule_num], axis=1)[1]
        else:
            c = softmax(b, 1)   
#        if i_activations is not None:
#            tf.transpose(tf.transpose(c, perm=[0,2,1]) * i_activations, perm=[0,2,1]) 
        outputs = K.batch_dot(c, u_hat_vecs, [2, 2])
        if i < iterations - 1:
            b = b + K.batch_dot(outputs, u_hat_vecs, [2, 3])                                    
    poses = squash_v1(outputs) 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations


In [ ]:
def vec_transformationByConv(poses, input_capsule_dim, input_capsule_num, output_capsule_dim, output_capsule_num):                            
    kernel = _get_weights_wrapper(
      name='weights', shape=[1, input_capsule_dim, output_capsule_dim*output_capsule_num], weights_decay_factor=0.0
    )
  
    u_hat_vecs = keras.backend.conv1d(poses, kernel)
    u_hat_vecs = keras.backend.reshape(u_hat_vecs, (-1, input_capsule_num, output_capsule_num, output_capsule_dim))
    u_hat_vecs = keras.backend.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
    return u_hat_vecs


In [ ]:
def vec_transformationByMat(poses, input_capsule_dim, input_capsule_num, output_capsule_dim, output_capsule_num, shared=False):                        
    inputs_poses_shape = poses.get_shape().as_list()
    poses = poses[..., tf.newaxis, :]        
    poses = tf.tile(poses, [1, 1, output_capsule_num, 1])    
    if shared:
        kernel = _get_weights_wrapper(name='weights', shape=[1, 1, output_capsule_num, output_capsule_dim, input_capsule_dim], weights_decay_factor=0.0)
        kernel = tf.tile(kernel, [inputs_poses_shape[0], input_capsule_num, 1, 1, 1])
    else:
        kernel = _get_weights_wrapper(name='weights', shape=[1, input_capsule_num, output_capsule_num, output_capsule_dim, input_capsule_dim], weights_decay_factor=0.0)
        kernel = tf.tile(kernel, [inputs_poses_shape[0], 1, 1, 1, 1])
    
    u_hat_vecs = tf.squeeze(tf.matmul(kernel, poses[...,tf.newaxis]),axis=-1)
    u_hat_vecs = keras.backend.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
    return u_hat_vecs

In [ ]:
def capsules_init(inputs, shape, strides, padding, pose_shape, add_bias, name):
    with tf.variable_scope(name):   
        poses = _conv2d_wrapper(
          inputs,
          shape=shape[0:-1] + [shape[-1] * pose_shape],
          strides=strides,
          padding=padding,
          add_bias=add_bias,
          activation_fn=None,
          name='pose_stacked'
        )        
        poses_shape = poses.get_shape().as_list()    
        poses = tf.reshape(poses, [-1, poses_shape[1], poses_shape[2], shape[-1], pose_shape])        
        beta_a = _get_weights_wrapper(name='beta_a', shape=[1, shape[-1]])    
        poses = squash_v1(poses, axis=-1)  
        activations = K.sqrt(K.sum(K.square(poses), axis=-1)) + beta_a        
        
  

    return poses, activations

In [ ]:
def capsule_fc_layer(nets, output_capsule_num, iterations, name, representation, args):
    with tf.variable_scope(name):   
        poses, i_activations = nets
        input_pose_shape = poses.get_shape().as_list()

        u_hat_vecs = vec_transformationByConv(poses,input_pose_shape[-1], input_pose_shape[1],input_pose_shape[-1], output_capsule_num,)
        
        beta_a = _get_weights_wrapper(name='beta_a', shape=[1, output_capsule_num])

        representation = tf.reshape(representation,[input_pose_shape[0],1,1,600])
        representation = tf.tile(representation,[1,args.num_classes,1,1])
        context = _get_weights_wrapper(name='context', shape=[input_pose_shape[0],args.num_classes,600,input_pose_shape[1]])
        context_sensitivity = tf.matmul(representation,context)
        context_sensitivity = tf.squeeze(context_sensitivity)

        poses, activations = routing(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations, context_sensitivity)
        
        
        
        
    return poses, activations

In [ ]:
def capsule_flatten(nets):
    poses, activations = nets
    input_pose_shape = poses.get_shape().as_list()
    
    poses = tf.reshape(poses, [
                    -1, input_pose_shape[1]*input_pose_shape[2]*input_pose_shape[3], input_pose_shape[-1]]) 
    activations = tf.reshape(activations, [
                    -1, input_pose_shape[1]*input_pose_shape[2]*input_pose_shape[3]])
    

    return poses, activations

In [ ]:
def capsule_conv_layer(nets, shape, strides, iterations, name, representation):   
    with tf.variable_scope(name):              
        poses, i_activations = nets
        
        inputs_poses_shape = poses.get_shape().as_list()

        hk_offsets = [
          [(h_offset + k_offset) for k_offset in range(0, shape[0])] for h_offset in
          range(0, inputs_poses_shape[1] + 1 - shape[0], strides[1])
        ]
        wk_offsets = [
          [(w_offset + k_offset) for k_offset in range(0, shape[1])] for w_offset in
          range(0, inputs_poses_shape[2] + 1 - shape[1], strides[2])
        ]
    
        inputs_poses_patches = tf.transpose(
          tf.gather(
            tf.gather(
              poses, hk_offsets, axis=1, name='gather_poses_height_kernel'
            ), wk_offsets, axis=3, name='gather_poses_width_kernel'
          ), perm=[0, 1, 3, 2, 4, 5, 6], name='inputs_poses_patches'
        )
        
    
        inputs_poses_shape = inputs_poses_patches.get_shape().as_list()
        inputs_poses_patches = tf.reshape(inputs_poses_patches, [
                                -1, shape[0]*shape[1]*shape[2], inputs_poses_shape[-1]
                                ])

        i_activations_patches = tf.transpose(
          tf.gather(
            tf.gather(
              i_activations, hk_offsets, axis=1, name='gather_activations_height_kernel'
            ), wk_offsets, axis=3, name='gather_activations_width_kernel'
          ), perm=[0, 1, 3, 2, 4, 5], name='inputs_activations_patches'
        )
        patches_dim = i_activations_patches.get_shape().as_list()
        
        i_activations_patches = tf.reshape(i_activations_patches, [
                                -1, shape[0]*shape[1]*shape[2]]
                                )
        u_hat_vecs = vec_transformationByConv(
                  inputs_poses_patches,
                  inputs_poses_shape[-1], shape[0]*shape[1]*shape[2],
                  inputs_poses_shape[-1], shape[3],
                  )
       

        # patches_dim = i_activations_patches.get_shape().as_list()
        representation = tf.expand_dims(representation, -1)
        representation = tf.tile(representation,[1,1,8])
        representation = tf.reshape(representation,[-1,1,16,300])
        representation = tf.tile(representation,[1,patches_dim[1],1,1])
        representation = tf.reshape(representation,[-1,16,300])
        context = _get_weights_wrapper(name='context', shape=[patches_dim[1]*patches_dim[0],300,48])
        context_sensitivity = tf.matmul(representation,context)
    
        beta_a = _get_weights_wrapper(
                name='beta_a', shape=[1, shape[3]]
                )
        poses, activations = routing(u_hat_vecs, beta_a, iterations, shape[3], i_activations_patches, context_sensitivity)
        poses = tf.reshape(poses, [
                    inputs_poses_shape[0], inputs_poses_shape[1],
                    inputs_poses_shape[2], shape[3],
                    inputs_poses_shape[-1]]
                ) 
        activations = tf.reshape(activations, [
                    inputs_poses_shape[0],inputs_poses_shape[1],
                    inputs_poses_shape[2],shape[3]]
                ) 
        nets = poses, activations            
   
    return nets

# **Load Data**

## From other repo

In [ ]:
import numpy as np
import os
import pickle
import pandas as pd


def checkdirs(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def read_text(filename):
    with open(filename, 'r') as f:
        best_epoch = int(f.read())
        return best_epoch


def write_text(text, filename):
    with open(filename, 'w') as f:
        f.write(text)


def load_pickle_data(pickle_dir, dataset):
    assert os.path.exists(pickle_dir)
    pickle_data = pickle.load(open(pickle_dir+dataset, "rb"))
    return pickle_data


def get_idx_from_sent(sent, word_idx_map, max_length):
    x = []
    words = sent.split()[:max_length]
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_length:
        x.append(0)
    return x


def make_idx_data(raw_datas, word_idx_map, len_train, max_length):
    data = []
    for raw_data in raw_datas:
        sent = get_idx_from_sent(raw_data["text"], word_idx_map, max_length)
        sent.append(raw_data["y"])
        data.append(sent)
    split = len_train
    train = np.array(data[:split], dtype="int")
    test = np.array(data[split:], dtype="int")
    return train, test


def preprocessing(data_path, dataset, long_sent=800):
    """

    :param data_path: base directory
    :param dataset: select dataset {'20news', 'mr', 'trec', 'mpqa'}
    :param long_sent: if dataset has long sentences, set to be constant length value
    :return: seq_length, num_classes, vocab_size, x_train, y_train, x_test, y_test, pre-train_word (GloVe 840b),
    word_idx
    """

    assert os.path.exists(data_path) is True

    x = load_pickle_data(data_path, dataset)
    data_frame, pretrain_word, len_train, n_exist_word, vocab, word_idx = x

    max_l = int(np.max(pd.DataFrame(data_frame)["num_words"]))

    if dataset in ["reuters", "20news", "imdb", 'mr']:
        train, test = make_idx_data(data_frame, word_idx, len_train, long_sent)
    else:
        train, test = make_idx_data(data_frame, word_idx, len_train, max_l)

    # train[:, :-1] = word idx
    # train[:, -1] = true label
    x_train = train[:, :-1]
    y_train = train[:, -1]

    x_test = test[:, :-1]
    y_test = test[:, -1]
    sequence_length = len(x_train[0])

    # make one-hot
    labels = sorted(list(set(y_train)))
    one_hot = np.zeros((len(labels), len(labels)), int)
    np.fill_diagonal(one_hot, 1)
    label_dict = dict(zip(labels, one_hot))

    y_train = np.eye(len(label_dict))[y_train]
    num_class = y_train.shape[1]

    y_test = np.eye(len(label_dict))[y_test]
    vocab_size = pretrain_word.shape[0]

    print("sequence length :", sequence_length)
    print("vocab size :", vocab_size)
    print("num classes :", num_class)

    return sequence_length, num_class, vocab_size, x_train, y_train, x_test, y_test, pretrain_word, word_idx


## load data

In [ ]:
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
  comment_texts.append(test_text)


  return comment_texts,comment_labels

In [ ]:
EMBEDDING_SIZE = 300 #@param [50, 150, 200, 250, 300, 350, 400, 450, 500]
embedding_type = "fasttext" #@param ["fasttext","word2vec"]
experiment_no = "2000" #@param [] {allow-input: true}

model_name = "DS_Caps_sinhala"

In [ ]:
folder_path =  '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/'

lankadeepa_data_path = folder_path + 'corpus/new/preprocess_from_isuru/lankadeepa_tagged_comments.csv'
gossip_lanka_data_path = folder_path + 'corpus/new/preprocess_from_isuru/gossip_lanka_tagged_comments.csv'
# "corpus/new/preprocess_from_unicode_values/lankadeepa_tagged_comments.csv"
# "corpus/new/preprocess_from_unicode_values/gossip_lanka_tagged_comments.csv"

context = 5
embeds = "fasttext"
word_embedding_path = folder_path + "word_embedding/"+embeds+"/source2_data_from_gosspiLanka_and_lankadeepa/"+str(EMBEDDING_SIZE)+"/"+embedding_type+"_"+str(EMBEDDING_SIZE)+"_"+str(context)
word_embedding_keydvectors_path = folder_path + "word_embedding/"+embeds+"/source2_data_from_gosspiLanka_and_lankadeepa/"+str(EMBEDDING_SIZE)+"/keyed_vectors/keyed.kv"
embedding_matrix_path = folder_path + 'Sentiment Analysis/CNN RNN/embedding_matrix/'+embedding_type+'_lankadeepa_gossiplanka_'+str(EMBEDDING_SIZE)+'_'+str(context)

experiment_name = folder_path + "Sentiment Analysis/CNN RNN/experiments/" +str(experiment_no) + "_"+ model_name +"_"+embedding_type+"_"+str(EMBEDDING_SIZE)+"_"+str(context)
model_save_path = folder_path + "Sentiment Analysis/CNN RNN/saved_models/"+str(experiment_no)+"_weights_best_"+model_name+"_"+embedding_type+"_"+str(experiment_no)+".hdf5"

In [ ]:
lankadeepa_data = pd.read_csv(lankadeepa_data_path)[:9059]
gossipLanka_data = pd.read_csv(gossip_lanka_data_path)
gossipLanka_data = gossipLanka_data.drop(columns=['Unnamed: 3'])

all_data = pd.concat([lankadeepa_data,gossipLanka_data], ignore_index=True)
all_data.shape

In [ ]:
all_data

In [ ]:
def text_preprocessing(data):
  comments = data['comment']
  labels = data['label']

  comments_splitted = []

  for comment in comments:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    comments_splitted.append(lines)

  return comments_splitted,labels

In [ ]:
comment_texts, comment_labels = text_preprocessing(all_data)

# prepare tokenizer

t = Tokenizer()
t.fit_on_texts(comment_texts)
vocab_size = len(t.word_index) + 1
print(vocab_size)

In [ ]:
comment_labels

In [ ]:
for i in range(len(comment_labels)):
  comment_labels[i] = comment_labels[i]-2

In [ ]:
comment_labels[15054]

In [ ]:
encoded_docs = t.texts_to_sequences(comment_texts)

In [ ]:
# max_length = len(max(encoded_docs, key=len))
max_length = 30
padded_docs = pad_sequences(encoded_docs, maxlen=max_length,padding='post')

comment_labels = np.array(comment_labels)
padded_docs = np.array(padded_docs)

In [ ]:
# comment_labels = pd.get_dummies(comment_labels).values
print('Shape of label tensor:', comment_labels.shape)
comment_labels = pd.get_dummies(comment_labels).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, comment_labels, test_size=0.1, random_state=0)

In [ ]:
y_train[0]

In [ ]:
(unique, counts) = np.unique(y_test, return_counts = True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

In [ ]:
X_train

In [ ]:
# !pip install gensim

In [ ]:
# import gensim
# from gensim.models.keyedvectors import KeyedVectors
# from gensim.models.fasttext import FastText
# from gensim.models import word2vec

In [ ]:
# word_embedding_model = FastText.load(word_embedding_path)

In [ ]:
# word_vectors = word_embedding_model.wv

In [ ]:
# word_vectors.vocab.items()

In [ ]:
# def generate_embedding_matrix():
#   if (embedding_type == 'fasText'):
#     word_embedding_model = FastText.load(word_embedding_path)
#   else:
#     word_embedding_model = word2vec.Word2Vec.load(word_embedding_path)
    
#   word_vectors = word_embedding_model.wv
#   word_vectors.save(word_embedding_keydvectors_path)
#   word_vectors = KeyedVectors.load(word_embedding_keydvectors_path, mmap='r')

#   embeddings_index = dict()
#   for word, vocab_obj in word_vectors.vocab.items():
#     embeddings_index[word]=word_vectors[word]

#   # create a weight matrix for words in training docs
#   embedding_matrix = zeros((vocab_size, EMBEDDING_SIZE))
#   for word, i in t.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#       embedding_matrix[i] = embedding_vector

#   # pickle.dump(embedding_matrix, open(embedding_matrix_path, 'wb'))
#   return embedding_matrix

In [ ]:
def load_word_embedding_matrix():
  f = open(embedding_matrix_path, 'rb')
  embedding_matrix= np.array(pickle.load(f))
  return embedding_matrix

In [ ]:
embedding_matrix = load_word_embedding_matrix()

# **Loss functions**

In [ ]:
def spread_loss(labels, activations, margin):
    activations_shape = activations.get_shape().as_list()
    mask_t = tf.equal(labels, 1)
    mask_i = tf.equal(labels, 0)    
    activations_t = tf.reshape(
      tf.boolean_mask(activations, mask_t), [activations_shape[0], 1]
    )    
    activations_i = tf.reshape(
      tf.boolean_mask(activations, mask_i), [activations_shape[0], activations_shape[1] - 1]
    )    
    gap_mit = tf.reduce_sum(tf.square(tf.nn.relu(margin - (activations_t - activations_i))))
    return gap_mit        

In [ ]:
def cross_entropy(y, preds):    
    y = tf.argmax(y, axis=1)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=preds, labels=y)                                               
    loss = tf.reduce_mean(loss) 
    return loss

In [ ]:
def margin_loss(y, preds):    
    y = tf.cast(y,tf.float32)
    loss = y * tf.square(tf.maximum(0., 0.9 - preds)) + \
        0.25 * (1.0 - y) * tf.square(tf.maximum(0., preds - 0.1))
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
#    loss = tf.reduce_mean(loss)    
    return loss

In [ ]:
def capsule_model_A(X, num_classes, args, context, context_sensitivity1):
    with tf.variable_scope('capsule_'+str(3), reuse=tf.AUTO_REUSE):   
        nets = _conv2d_wrapper(
                X, shape=[3, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
      
        nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                             padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
        nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2', representation=context_sensitivity1)
        nets = capsule_flatten(nets)
        poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2', context, args) 
    return poses, activations

In [ ]:
def capsule_model_B(X, num_classes, args, representation1, representation2):
    poses_list = []
    for _, ngram in enumerate([3,4,5]):
        with tf.variable_scope('capsule_'+str(ngram), reuse=tf.AUTO_REUSE): 
            nets = _conv2d_wrapper(
                X, shape=[ngram, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
         
            nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                                 padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
            nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2', representation=representation1)
            nets = capsule_flatten(nets)
            poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2', representation2, args)
            poses_list.append(poses)
    
    poses = tf.reduce_mean(tf.convert_to_tensor(poses_list), axis=0) 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations

# Set Config

## Config Capsule

In [ ]:
class Args:
  embedding_type = "static"
  dataset = "reuters_multilabel_dataset"
  loss_type = "margin_loss"
  model_type = "capsule-B"
  has_test = 1
  has_dev = 1
  num_epochs = 50
  batch_size = 4
  use_orphan = True
  use_leaky = True
  learning_rate = 0.001
  margin = 0.2
  num_classes = 4
  vocab_size = vocab_size
  vec_size = 300
  max_sent = max_length


In [ ]:
args = Args()

## Config SLSTM

In [ ]:
class Config(object):
    vocab_size=vocab_size
    max_grad_norm = 5
    init_scale = 0.05
    hidden_size = 300
    lr_decay = 0.95
    valid_portion = 0.0
    batch_size = 16
    keep_prob = 0.8
    #0.05
    learning_rate = 0.0001
    max_epoch =2
    # max_max_epoch =40
    max_max_epoch = 15
    num_label=5
    attention_iteration=3
    random_initialize=True
    embedding_trainable=True
    l2_beta=0.0
    layer = 4
    step = 1

In [ ]:
config = Config()

# Tensorflow Graph

In [ ]:
# X = tf.placeholder(tf.int32, [args.batch_size, args.max_sent], name="input_x")
# y = tf.placeholder(tf.int64, [args.batch_size, args.num_classes], name="input_y")
# is_training = tf.placeholder_with_default(False, shape=())    
# learning_rate = tf.placeholder(dtype='float32') 
# margin = tf.placeholder(shape=(),dtype='float32') 

# l2_loss = tf.constant(0.0)

In [ ]:
# print(y)

In [ ]:
# w2v = np.array(embedding_matrix,dtype=np.float32)

In [ ]:
# w2v.shape

In [ ]:
# W1 = tf.Variable(w2v, trainable = False)
# X_embedding = tf.nn.embedding_lookup(W1, X)
# X_embedding = X_embedding[...,tf.newaxis] 

In [ ]:
# tf.logging.info("input dimension:{}".format(X_embedding.get_shape()))

In [ ]:
# X_embedding.shape

In [ ]:
# classifier = Classifer(config)

In [ ]:
# hidden_states,representation, representation1 = classifier.slstm_basic_layer(X_embedding, config, [max_length]*(args.batch_size))

In [ ]:
# hidden_states.shape

In [ ]:
# representation1.shape

In [ ]:
# representation1 = tf.expand_dims(representation1, -1)

In [ ]:
# representation1 = tf.tile(representation1,[1,1,2])

In [ ]:
# representation1.shape

In [ ]:
# representation1 = tf.reshape(representation1,[-1,1,16,75])

In [ ]:
# representation1.shape

In [ ]:
# representation1 = tf.tile(representation1,[1,23,1,1])

In [ ]:
# representation1.shape 

In [ ]:
# representation1 = tf.reshape(representation1,[-1,16,75])

In [ ]:
# representation1.shape

In [ ]:
# context1 = tf.Variable(tf.random_normal([92,75,48], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32)

In [ ]:
# context_sensitivity1 = tf.matmul(representation1,context1)

In [ ]:
# context_sensitivity1.shape

In [ ]:
# context_sensitivity1 = tf.reshape(context_sensitivity1,[92,16,48])

In [ ]:
# context_sensitivity1.shape

In [ ]:
# hidden_states = tf.expand_dims(hidden_states, -1)

In [ ]:
# hidden_states.shape

In [ ]:
# hidden_states.shape[0:-1]

In [ ]:
# representation = tf.reshape(representation,[4,2,300,1])

In [ ]:
# representation.shape

In [ ]:
# context = tf.Variable(tf.random_normal([4,2,368,300], mean=0.0, stddev=0.1, dtype=tf.float32), dtype=tf.float32)

In [ ]:
# context_sensitivity = tf.matmul(context,representation)

In [ ]:
# context_sensitivity = tf.squeeze(context_sensitivity)

In [ ]:
# context_sensitivity.shape

In [ ]:
# poses, activations = capsule_model_A(X_embedding, args.num_classes)   
# poses, activations = capsule_model_B(hidden_states, args.num_classes, args, representation, representation1)   

In [ ]:
# loss = margin_loss(y, activations) 

In [ ]:
# y_pred = tf.argmax(activations, axis=1, name="y_proba")    
# correct = tf.equal(tf.argmax(y, axis=1), y_pred, name="correct")
# accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)   
# training_op = optimizer.minimize(loss, name="training_op")
# gradients, variables = zip(*optimizer.compute_gradients(loss))

In [ ]:
# with tf.device('/cpu:0'):
#     global_step = tf.train.get_or_create_global_step()

In [ ]:
args.max_sent = max_length
threshold = 0.5

In [ ]:
# grad_check = [tf.check_numerics(g, message='Gradient NaN Found!')
#               for g in gradients if g is not None] + [tf.check_numerics(loss, message='Loss NaN Found')]
# with tf.control_dependencies(grad_check):
#     training_op = optimizer.apply_gradients(zip(gradients, variables), global_step=global_step)      

In [ ]:
# sess = tf.InteractiveSession()
# from keras import utils

In [ ]:
class BatchGenerator(object):
    """Generate and hold batches."""
    def __init__(self, dataset,label, batch_size,input_size, is_shuffle=True):
      self._dataset = dataset
      self._label = label
      self._batch_size = batch_size    
      self._cursor = 0      
      self._input_size = input_size      
      
      if is_shuffle:
          index = np.arange(len(self._dataset))
          np.random.shuffle(index)
          self._dataset = np.array(self._dataset)[index]
          self._label = np.array(self._label)[index]
      else:
          self._dataset = np.array(self._dataset)
          self._label = np.array(self._label)
    def next(self):
      if self._cursor + self._batch_size > len(self._dataset):
          self._cursor = 0
      """Generate a single batch from the current cursor position in the data."""      
      batch_x = self._dataset[self._cursor : self._cursor + self._batch_size,:]
      batch_y = self._label[self._cursor : self._cursor + self._batch_size]
      self._cursor += self._batch_size
      return batch_x, batch_y

In [ ]:
# n_iterations_per_epoch = len(X_train) // args.batch_size
# n_iterations_test = len(X_test) // args.batch_size
# n_iterations_dev = len(X_test) // args.batch_size    

# mr_train = BatchGenerator(X_train,y_train, args.batch_size, 0)    
# mr_dev = BatchGenerator(X_test,y_test, args.batch_size, 0)
# mr_test = BatchGenerator(X_test,y_test, args.batch_size, 0, is_shuffle=False)

best_model = None
best_epoch = 0
best_acc_val = 0.

In [ ]:
# init = tf.global_variables_initializer()
# sess.run(init)     

lr = args.learning_rate
m = args.margin

In [ ]:
# array1 = [2,3,0,1]
# z = utils.to_categorical(array1, 4)

In [ ]:
# z

# Train/ Validate

In [ ]:
# loss_vals, acc_vals = [], []
# for epoch in range(args.num_epochs):
#     for iteration in range(1, n_iterations_per_epoch + 1):                
#         X_batch, y_batch = mr_train.next()     
#         y_batch = utils.to_categorical(y_batch, args.num_classes)        
#         _, loss_train, probs, capsule_pose = sess.run(
#             [training_op, loss, activations, poses],
#             feed_dict={X: X_batch[:,:args.max_sent],
#                        y: y_batch,
#                        is_training: True,
#                        learning_rate:lr,
#                        margin:m})        
#         print("\rIteration: ({:.1f}%)  Loss: {:.5f}".format(
#                   iteration * 100 / n_iterations_per_epoch,
#                   loss_train),
#               end="")                        
#     loss_vals, acc_vals = [], []
#     for iteration in range(1, n_iterations_dev + 1):
#         X_batch, y_batch = mr_dev.next()            
#         y_batch = utils.to_categorical(y_batch, args.num_classes)
#         loss_val, acc_val = sess.run(
#                 [loss, accuracy],
#                 feed_dict={X: X_batch[:,:args.max_sent],
#                            y: y_batch,
#                            is_training: False,
#                            margin:m})
#         loss_vals.append(loss_val)
#         acc_vals.append(acc_val)
#     loss_val, acc_val = np.mean(loss_vals), np.mean(acc_vals)    
#     print("\rEpoch: {}  Val accuracy: {:.1f}%  Loss: {:.4f}".format(
#         epoch + 1, acc_val * 100, loss_val))
               
#     # preds_list, y_list = [], []
#     # for iteration in range(1, n_iterations_test + 1):
#     #     X_batch, y_batch = mr_test.next()             
#     #     probs = sess.run([activations],
#     #             feed_dict={X:X_batch[:,:args.max_sent],
#     #                        is_training: False})
#     #     preds_list = preds_list + probs[0].tolist()
#     #     y_list = y_list + y_batch.tolist()
        
#     # y_list = np.array(y_list)
#     # preds_probs = np.array(preds_list)                
#     # preds_probs[np.where( preds_probs >= threshold )] = 1.0
#     # preds_probs[np.where( preds_probs < threshold )] = 0.0 
#     # print(y_list)
#     # print(preds_probs)
    
#     # #[precision, recall, F1, support] = precision_recall_fscore_support(y_list, preds_probs, average='samples')
#     # acc = accuracy_score(y_list, preds_probs)

#     # print ('\rER: %.3f' % acc, 'Precision: %.3f' % precision, 'Recall: %.3f' % recall, 'F1: %.3f' % F1)  
#     # # if args.model_type == 'CNN' or args.model_type == 'KIMCNN':
#     # #     lr = max(1e-6, lr * 0.8)
#     if args.loss_type == 'margin_loss':    
#         m = min(0.9, m + 0.1)
# print(acc_vals)

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, precision_recall_fscore_support

In [ ]:
acc_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []

kfold = KFold(n_splits=10, shuffle=True)

fold_no = 1
inputs = padded_docs
targets = comment_labels

for train, test in kfold.split(inputs, targets):
  

    n_iterations_per_epoch = len(inputs[train]) // args.batch_size
    n_iterations_test = len(inputs[test]) // args.batch_size

    mr_train1 = BatchGenerator(inputs[train], targets[train], args.batch_size, 0)    
    mr_test1 = BatchGenerator(inputs[test], targets[test], args.batch_size, 0, is_shuffle=False)
    best_accuracy = 0.
    best_precision = 0.
    best_recall = 0.
    best_f1 = 0.

    X = tf.placeholder(tf.int32, [args.batch_size, args.max_sent], name="input_x")
    y = tf.placeholder(tf.int64, [args.batch_size, args.num_classes], name="input_y")
    is_training = tf.placeholder_with_default(False, shape=())    
    learning_rate = tf.placeholder(dtype='float32') 
    margin = tf.placeholder(shape=(),dtype='float32') 

    l2_loss = tf.constant(0.0)
    w2v = np.array(embedding_matrix,dtype=np.float32)

    W1 = tf.Variable(w2v, trainable = False)
    X_embedding = tf.nn.embedding_lookup(W1, X)
    X_embedding = X_embedding[...,tf.newaxis] 

    classifier = Classifer(config)
    hidden_states,representation, representation1 = classifier.slstm_basic_layer(X_embedding, config, [max_length]*(args.batch_size))
    hidden_states = tf.expand_dims(hidden_states, -1)


    poses, activations = capsule_model_A(hidden_states, args.num_classes, args, representation, representation1)   
    loss = margin_loss(y, activations) 
    y_pred = tf.argmax(activations, axis=1, name="y_proba")    
    correct = tf.equal(tf.argmax(y, axis=1), y_pred, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name = 'opt'+str(fold_no))   
    training_op = optimizer.minimize(loss, name="training_op")
    gradients, variables = zip(*optimizer.compute_gradients(loss)) 
    with tf.Session() as sess:

      init = tf.global_variables_initializer()
      sess.run(init)

      for epoch in range(0,6): 

        for iteration in range(1, n_iterations_per_epoch + 1):
            
          
          X_batch, y_batch = mr_train1.next()          
          _, loss_train, probs, capsule_pose = sess.run(
                [training_op, loss, activations, poses],
                feed_dict={X: X_batch[:,:args.max_sent],
                          y: y_batch,
                          is_training: True,
                          learning_rate:lr,
                          margin:m})        
          print("\rIteration: {}/{} ({:.1f}%) epoch:{}  Loss: {:.5f}".format(iteration, n_iterations_per_epoch, iteration * 100 / n_iterations_per_epoch, epoch+1, loss_train), end="") 
          # print("\r ({}) epoch:{}".format( 'running', epoch+1), end="")                        
        preds_list, y_list = [], []
        for iteration in range(1, n_iterations_test + 1):
          X_batch, y_batch = mr_test1.next()             
          probs = sess.run([activations],
                    feed_dict={X:X_batch[:,:args.max_sent],
                                is_training: False})
          preds_list = preds_list + probs[0].tolist()
          y_list = y_list + y_batch.tolist()

        y_list = np.array(y_list)
        preds_probs = np.array(preds_list)  
        labels = np.argmax(y_list, axis=1)
        predictions = np.argmax(preds_probs, axis=1)

        accuracy_fold = accuracy_score(labels, predictions)
        precision_fold = precision_score(labels, predictions, average='weighted', zero_division = 0 )
        recall_fold = recall_score(labels, predictions, average='weighted')
        f1_fold = f1_score(labels, predictions, average='weighted')
        if best_f1 <= f1_fold :
          best_accuracy = accuracy_fold
          best_precision = precision_fold
          best_recall = recall_fold
          best_f1 = f1_fold
        

      acc_per_fold.append(best_accuracy)
      precision_per_fold.append(best_precision)
      recall_per_fold.append(best_recall)
      f1_per_fold.append(best_f1)
      print("\rFold: {} accuracy: {:.4f}%  Precision: {:.4f} recall: {:.4f} F1: {:.4f}".format(fold_no, best_accuracy, best_precision, best_recall, best_f1))
      if args.loss_type == 'margin_loss':    
            m = min(0.9, m + 0.1)
      fold_no += 1

accuracy = np.mean(acc_per_fold)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = np.mean(precision_per_fold)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = np.mean(recall_per_fold)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = np.mean(f1_per_fold)
print('F1 score: %f' % f1)

# Test Model

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
preds_list, y_list = [], []
for iteration in range(1, n_iterations_test + 1):
    X_batch, y_batch = mr_test.next()             
    probs = sess.run([activations],
            feed_dict={X:X_batch[:,:args.max_sent],
                        is_training: False})
    preds_list = preds_list + probs[0].tolist()
    y_list = y_list + y_batch.tolist()
        
y_list = np.array(y_list)
preds_probs = np.array(preds_list)                
preds_probs[np.where( preds_probs >= threshold )] = 1.0
preds_probs[np.where( preds_probs < threshold )] = 0.0 
    
# [precision, recall, F1, support] = precision_recall_fscore_support(y_list, preds_probs, average='samples')
# acc = accuracy_score(y_list, preds_probs)

In [ ]:
print(len(preds_list))
pred1 = preds_probs[:,1].tolist()
pred11 = [int(i) for i in pred1]

In [ ]:
y_list1 = y_list.tolist()

In [ ]:
# y_list1

In [ ]:
accuracy = accuracy_score(y_list1, pred11)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_list1, pred11)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_list1, pred11)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_list1, pred11)
print('F1 score: %f' % f1)

In [ ]:
y_list

In [ ]:
best_acc_val

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers[torch]

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
folder_path = '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Bert/'
folder_path_data =  '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/'

lankadeepa_data_path = folder_path_data + 'corpus/new/preprocess_from_isuru/lankadeepa_tagged_comments.csv'
gossip_lanka_data_path = folder_path_data + 'corpus/new/preprocess_from_isuru/gossip_lanka_tagged_comments.csv'

In [ ]:
def embeds(model_output):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    return token_embeddings

In [ ]:
tokenizer = BertTokenizer.from_pretrained(folder_path + 'xnli_output')
model = BertModel.from_pretrained(folder_path + 'xnli_output', return_dict=True)

In [ ]:
import pandas as pd
lankadeepa_data = pd.read_csv(lankadeepa_data_path)[:9059]
gossipLanka_data = pd.read_csv(gossip_lanka_data_path)
gossipLanka_data = gossipLanka_data.drop(columns=['Unnamed: 3'])

all_data = pd.concat([lankadeepa_data,gossipLanka_data], ignore_index=True)

In [ ]:
sentences = all_data.comment

In [ ]:
list_of_sentences = []
for i in sentences:
  list_of_sentences.append(i)

In [ ]:
encoded_input = tokenizer(list_of_sentences, padding=True, truncation=True, max_length=20, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = embeds(model_output)

In [ ]:
type(sentence_embeddings)

In [ ]:
Numpy_embedss = sentence_embeddings.numpy()

In [ ]:
type(Numpy_embedss)

In [ ]:
!cd '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Bert'

In [ ]:
import numpy as np

np.save('context_embeds', Numpy_embedss)

In [ ]:
import pickle as pkl

with open("contextual_embeds", 'wb') as file1:
  pkl.dump(Numpy_embedss, file1)
  


In [ ]:
!cp  -r  '/content/context_embeds.npy' '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Bert'

In [ ]:
!cp  -r  '/content/contextual_embeds' '/content/drive/My Drive/Final Year Project/FYP/Sentiment Analysis/Implementation/Sentiment Analysis/Bert'